### 기본 세팅

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import time
import pandas as pd
import re

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# driver = webdriver.Chrome()

In [2]:
def scroll_down(driver, scroll_ratio): 
    total_height = driver.execute_script("return document.body.scrollHeight") # 현재 문서의 전체 높이 가져오기
    scroll_position = total_height * scroll_ratio # 스크롤할 위치 계산
    driver.execute_script(f"window.scrollTo(0, {scroll_position});") # 스크롤 실행
    time.sleep(2) # 짧은 시간 대기

In [3]:
def review_scroller(scroll):
    scroll_attempts = 0  # 스크롤 시도 횟수
    max_attempts = 5     # 최대 시도 횟수
    number = 0

    while True:
        number = number + 1
        print(number)  # 스크롤 횟수

        # 스크롤 전 문서 높이
        last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)  # 현재 문서의 높이
        print(f'last height: {last_height}')

        # 선택한 요소를 기준으로 페이지를 10000 픽셀 아래로 스크롤
        driver.execute_script('arguments[0].scrollBy(0, 10000);', scroll)
        time.sleep(2)  # 스크롤 후 대기

        # 스크롤 후 문서 높이
        new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
        print(f'new height: {new_height}')

        if new_height == last_height:
            scroll_attempts += 1
        else:
            scroll_attempts = 0

        if scroll_attempts >= max_attempts:
            break

In [4]:
driver.maximize_window() 

In [ ]:
driver.quit()

## 영화 정보 페이지

In [10]:
def get_movie_info(keyword):
    search_keyword = "영화 " + keyword 
    base_url = "https://search.naver.com/search.naver?where=view&sm=tab_jum&query="
    search_url = base_url + search_keyword
    driver.get(search_url)
    time.sleep(3)

    try:
        specific = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[1]/section[1]/div/div/div')
        if '제외한 상세검색 결과입니다' in specific.text:
            driver.find_element(By.CSS_SELECTOR, 'a.more_link').click()
            print("일반 결과 클릭")
            time.sleep(2)
    except Exception as e:
        pass

    #정보 창으로 이동 
    try:
        for j in range(5): 
            element = driver.find_elements(By.CSS_SELECTOR, 'span.menu')[j]
            if '기본정보' in element.text:
                element.click()
                time.sleep(2)
    except Exception as e:
        pass

    def get_text_or_na(xpath):
        try:
            element = driver.find_element(By.XPATH, xpath)
            return element[0].text
        except Exception:
            return "NA"

    # Age rating
    grade = get_text_or_na('//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/dl/div[2]/dd')
    
    # Genre
    genre = get_text_or_na('//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/dl/div[3]/dd')
    
    # Running time
    running_time = get_text_or_na('//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/dl/div[5]/dd')
    
    # Add elements to list
    add_elements = [keyword, grade, genre, running_time]
    return add_elements


In [ ]:
movie_list = []

In [13]:
i = 0
for movie in movie_list:
    movie_info_df = pd.DataFrame(columns=['keyword','grade','genre','running_time'])
    add_elements = get_movie_info(movie)
    print(f'{i+1}th movie info crawled') 
    print(add_elements)
    movie_info_df.loc[i] = add_elements
    i += 1
    movie_info_df.to_csv(f'/Users/yewonsong/desktop/info/2016_movie_info.csv',index=True)

1 movie info crawled
['인사이드아웃2', '전체 관람가', '애니메이션', '96분', '켈시 맨\n감독', ['에이미 포엘러', '마야 호크', '루이스 블랙', '필리스 스미스', '토니 헤일']]
2 movie info crawled
['이프', '전체 관람가', '코미디, 드라마, 가족', '104분', '존 크래신스키\n감독', ['라이언 레이놀즈', '케일리 플레밍', '존 크래신스키', '피오나 쇼우', '바비 모니한', '스티브 카렐', '아콰피나']]


In [12]:
movie_info_df 

,keyword,grade,genre,running_time,director,actors_list
0,인사이드아웃2,전체 관람가,애니메이션,96분,켈시 맨\n감독,"[에이미 포엘러, 마야 호크, 루이스 블랙, 필리스 스미스, 토니 헤일]"
1,이프,전체 관람가,"코미디, 드라마, 가족",104분,존 크래신스키\n감독,"[라이언 레이놀즈, 케일리 플레밍, 존 크래신스키, 피오나 쇼우, 바비 모니한, 스..."
